<a href="https://colab.research.google.com/github/chejige/Youtube-view-Estimator-and-dashboard/blob/main/Dashboard(streamlit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 4.0 MB/s 
     |████████████████████████████████| 235 kB 48.4 MB/s 
     |████████████████████████████████| 181 kB 73.6 MB/s 
     |████████████████████████████████| 164 kB 47.4 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 49.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=22767183c43395000060992adc1bed785ffda3189743cf3deafcf4375cdaa9d4
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
# import files
# import relevant libraries (visualization, dashboard, data manipulation)
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px
import streamlit as st
from datetime import datetime

In [ ]:
# libaries for load csv data
#import lib for upload CSV files
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
 
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
#Define Functions 
# load data

def load_data():
    link = 'https://drive.google.com/file/d/1-HBoI3xKTxqTf7Jyx_Ut0-aZjwMlv910/view?usp=sharing'
 
    # to get the id part of the file
    id = link.split("/")[-2]
 
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('video__link.csv') 
    df = pd.read_csv('video__link.csv')
    return df

In [ ]:
df=load_data()

In [ ]:
  
df=df.drop(['Unnamed: 0','description'],axis=1)

In [ ]:
df.dtypes


ID               object
Title            object
Publish_date     object
viewer_count      int64
like_count        int64
comment_count     int64
categroyID        int64
duration         object
Links            object
dtype: object

In [ ]:
#build dashboard
add_sidebar = st.sidebar.selectbox('Aggregate or Individual Video', ('Aggregate Metrics','Individual Video Analysis','data anylysis'))

In [ ]:
df['Publish_date']=df['Publish_date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [ ]:

if add_sidebar == 'data anylysis':
    
    metric_date_12mon=df['Publish_date'].max()- pd.DateOffset(months=12)
    st.write("'StatQuest with Josh Starmer' youtube channel data analysis ")
    st.write("Updated until: ", df['Publish_date'].max())
    
    
    Total_viewer=df['viewer_count'].sum()
    Total_videos=df['ID'].count().sum()
    Average_viewer=round(df['viewer_count'].sum()/df['ID'].count().sum(),1)
    col1, col2, col3 = st.columns(3)
    col1.metric('Total viewer',Total_viewer)
    col2.metric('Total videos',Total_videos)
    col3.metric('Average viewers',Average_viewer)
    Top_like_data=df.sort_values('like_count',ascending=False).iloc[:10,:]
    st.write('Top 10 like videos: ',Top_like_data.iloc[:,1:])
#like count in recent 1 year
    data_12moth=df[df['Publish_date']>=metric_date_12mon]
    fig = px.bar(data_12moth, x ='Publish_date', y='like_count')
    #order axis 
    st.plotly_chart(fig)
    fig2 = px.bar(data_12moth, x ='Publish_date', y='viewer_count')
    #order axis 
    st.plotly_chart(fig2)

In [ ]:
if add_sidebar == 'Individual Video Analysis':
    videos = tuple(df['Title'])
    st.write("Individual Video Performance")
    video_select = st.selectbox('Pick a Video:', videos)
    
    agg_filtered = df[df['Title'] == video_select]
    video_date=agg_filtered['Publish_date']
    st.write(agg_filtered.T)
    